In [1]:
import os
import imageio_ffmpeg
os.environ["IMAGEIO_FFMPEG_EXE"] = "C:\\ffmpeg\\bin\\ffmpeg.exe"

In [2]:
import torch
from transformers import pipeline
from pydub import AudioSegment
import speech_recognition as sr
from konlpy.tag import Okt
from gensim import corpora, models
import re
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import tempfile
from google.cloud import storage, speech_v1p1beta1 as speech
import moviepy.editor as mp
from moviepy.editor import VideoFileClip, ImageSequenceClip
import kss
import dlib
import cv2
import numpy as np
from tqdm import tqdm


In [8]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "interview-400010-9abf1ded3113.json"

# Step 0: 영상에서 음성 추출
def extract_audio_from_video(video_file, audio_file):
    video = VideoFileClip(video_file)
    video.audio.write_audiofile(audio_file)
    print(f"Audio extracted to {audio_file}")

# Step 1: 음성 파일에서 텍스트 추출
def audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio_file)
    audio = audio.set_channels(1)  # 모노로 변환
    audio.export("temp.wav", format="wav")

    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)

    client = speech.SpeechClient()

    # 오디오 파일의 샘플링 속도 확인
    audio = AudioSegment.from_wav("temp.wav")
    sample_rate = audio.frame_rate

    # 오디오 파일을 청크로 나누기
    chunk_length_ms = 10000  # 10초 단위로 청크 나누기
    chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

    transcript = ""

    for i, chunk in enumerate(chunks):
        chunk.export(f"chunk{i}.wav", format="wav")
        with open(f"chunk{i}.wav", "rb") as audio_file:
            content = audio_file.read()
        audio = speech.RecognitionAudio(content=content)
        config = speech.RecognitionConfig(
            encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=sample_rate,
            language_code="ko-KR",
        )

        try:
            response = client.recognize(config=config, audio=audio)
            transcript += "".join([result.alternatives[0].transcript for result in response.results])
        except Exception as e:
            print(f"Google Cloud Speech-to-Text API request error: {e}")
            raise ValueError(f"Could not request results from Google Cloud Speech-to-Text service; {e}")
        finally:
            os.remove(f"chunk{i}.wav")

    os.remove("temp.wav")
    print("Transcript extracted")
    return transcript

# Step 2: 텍스트 전처리 및 문장 분리
def preprocess_text(text):
    # 한글과 공백만 남기기
    processed_text = re.sub(r'[^가-힣\s]', '', text)
    # KSS를 사용하여 문장 단위로 분리
    sentences = kss.split_sentences(processed_text)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    print(f"Number of sentences: {len(sentences)}")
    return sentences

# Step 3: 주제 추출 및 요약
def extract_topics_and_summarize(sentences, num_topics=5):
    if not sentences or len(sentences) < 2:
        return {"error": "Not enough data to extract topics. Please provide more text."}

    # 문장 임베딩 모델 로드
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    sentence_embeddings = model.encode(sentences)

    # Ensure num_topics is not greater than the number of unique sentences
    num_topics = min(num_topics, len(sentences))

    if num_topics < 2:
        return {"error": "Not enough data to extract topics."}

    # AgglomerativeClustering 클러스터링 수행
    clustering_model = AgglomerativeClustering(n_clusters=num_topics)
    clustering_model.fit(sentence_embeddings)
    cluster_assignment = clustering_model.labels_

    # 클러스터별 문장 그룹화
    clustered_sentences = {i: [] for i in range(num_topics)}
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(sentences[sentence_id])

    # 클러스터별로 요약 수행
    summarizer = pipeline("summarization", model="gogamza/kobart-base-v2", tokenizer="gogamza/kobart-base-v2")
    topic_sentences = {}
    for i, sentences in clustered_sentences.items():
        combined_text = ' '.join(sentences)
        if len(combined_text.split()) > 50:  # 50 단어 이상인 경우에만 요약 수행
            summary = summarizer(combined_text, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
        else:
            summary = combined_text
        topic_sentences[f'주제 {i+1}'] = summary

    print("Topics extracted and summarized")
    return topic_sentences

# 주제 관련 문장 찾기
def find_relevant_sentences(sentences, topic_summary, max_duration=60):
    okt = Okt()
    topic_nouns = okt.nouns(topic_summary)
    relevant_sentences = [sentence for sentence in sentences if any(noun in sentence for noun in topic_nouns)]

    total_duration = 0
    relevant_text = []

    for sentence in relevant_sentences:
        duration = len(sentence.split()) / 2.5  # 대략적으로 초당 2.5 단어로 가정
        if total_duration + duration > max_duration:
            break
        relevant_text.append(sentence)
        total_duration += duration

    return relevant_text, total_duration

# 중요 프레임 추출 (1분 내의 영상만을 대상으로)
def extract_important_frames(video_file, start_time, end_time, sample_every_sec=2):
    detector = dlib.get_frontal_face_detector()
    cap = cv2.VideoCapture(video_file)

    frames = []
    timestamps = []
    face_centers = []

    frame_count = 0
    total_frames = int((end_time - start_time) * cap.get(cv2.CAP_PROP_FPS))
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)  # 영상의 시작 시간 설정

    with tqdm(total=total_frames // sample_every_sec) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
            if timestamp > end_time:
                break
            if int(timestamp - start_time) % sample_every_sec == 0:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = detector(gray)
                if faces:
                    face = faces[0]
                    center_x = (face.left() + face.right()) // 2
                    face_centers.append(center_x)
                else:
                    face_centers.append(frame.shape[1] // 2)  # 얼굴을 찾지 못한 경우 중앙을 중심으로 설정
                frames.append(frame)
                timestamps.append(timestamp)
                # 메모리 해제
                del frame
                frame_count += 1
                pbar.update(1)
                if frame_count % 100 == 0:  # 메모리 해제 빈도를 조정
                    cv2.waitKey(1)
    cap.release()

    print(f"Extracted face centers: {face_centers}")  # Debugging line
    return timestamps, face_centers

# 영상 세로로 자르기
def cut_video_with_focus(video_file, start_time, end_time, output_file, focus_frames):
    video = VideoFileClip(video_file).subclip(start_time, end_time)

    # 영상 크기 조정 (세로 비율 9:16)
    target_height = 720  # 해상도 줄이기
    video = video.resize(height=target_height)

    def crop_center(image, center_x):
        print(f"Received center_x: {center_x}")  # Debugging line
        height, width, _ = image.shape
        new_width = int(height * 9 / 16)
        left = max(0, min(center_x - new_width // 2, width - new_width))
        print(f"Cropping image: height={height}, width={width}, center_x={center_x}, new_width={new_width}, left={left}")  # Debugging line
        cropped_image = image[:, left:left + new_width]
        return cropped_image

    # 임시 디렉토리 생성
    with tempfile.TemporaryDirectory() as temp_dir:
        frame_files = []

        for t, frame in enumerate(video.iter_frames()):
            idx = int((t / len(focus_frames)) * len(focus_frames))
            idx = min(idx, len(focus_frames) - 1)
            center_x = focus_frames[idx]
            print(f"Time: {t}, Index: {idx}, Center X: {center_x}")  # Debugging line
            cropped_frame = crop_center(frame, center_x)

            # 임시 파일에 프레임 저장
            frame_file = os.path.join(temp_dir, f"frame_{t:04d}.png")
            try:
                cv2.imwrite(frame_file, cv2.cvtColor(cropped_frame, cv2.COLOR_RGB2BGR))
                if os.path.exists(frame_file):
                    frame_files.append(frame_file)
                else:
                    print(f"Failed to save frame {t} to {frame_file}")
            except Exception as e:
                print(f"Error saving frame {t}: {e}")

        # 프레임 파일 리스트 확인
        if not frame_files:
            print("No frames were processed.")
            return

        # ImageSequenceClip을 사용하여 프레임 파일로부터 동영상 생성
        try:
            new_clip = ImageSequenceClip(frame_files, fps=video.fps)

            # 원본 비디오에서 오디오 트랙 추출
            audio = video.audio
            new_clip = new_clip.set_audio(audio)

            new_clip.write_videofile(output_file, codec='libx264', audio_codec='aac')
        except Exception as e:
            print(f"Error creating video from frames: {e}")


In [4]:
video_file = 'videos/news1.mp4'  # 비디오 파일 경로 설정
audio_file = 'videos/extracted_audio.wav'

# Step 0: 영상에서 음성 추출
extract_audio_from_video(video_file, audio_file)
print("Step 0: 음성 추출 완료")

try:
    transcript = audio_to_text(audio_file)
    print("추출된 텍스트:", transcript)
except Exception as e:
    print(f"Error: {e}")

MoviePy - Writing audio in videos/extracted_audio.wav


MoviePy - Done.
Audio extracted to videos/extracted_audio.wav
Step 0: 음성 추출 완료
Transcript extracted
추출된 텍스트: 여러분 안녕하십니까 오늘은 저희가 단독 취재한 내용으로 뉴스 시작하겠습니다 다음 주 월요일부터는 병원에 갈 때신분증을 꼭 챙겨야 합니다 환자가 건강보험 자격이 있는지 혹시 다른 사람 명의로 진료받는 건 아닌지 병원이 확인하기 위해서입니다신분증 대신 휴대전화에 모바일 건강보험증을 설치해서 그걸 병원에 보여 줘도 됩니다 그런데이 모바일다른 사람의 휴대전화 해도 쉽게 설치할 수 있고 또 그걸 병원이 적발하기 어렵다는 사실이 저희 취재 결과 확인됐습니다기자의 단독 보도입니다 신분증 지참 필수 란 안내 포스터가 붙은 한내과의원 의원 협조를 미리 받아모바일 건강보험증으로 진료 접수를 해 봤습니다 건강보험증 qr 코드를 병원 기기로 인식하자 건강 보험 자격 확인이 되고 문제 없이완료됐습니다 하지만 기자가 제시한 모바일 건강보험증은 본인이 아닌 동료의 것입니다 타인 명의의 건강보험증을 병원에 제출했는데걸러내지 못한 겁니다 어떻게 이게 가능한걸까 모바일 건강보험증 앱은 휴대 전화 번호를 입력한뒤 인증번호를 받아 본인임을 확인하는설치할 수 있습니다 그런데 두 사람이 미리 짜고 상대방 휴대전화에 인증번호를 받을 사람의 전화 번호를 입력하고그렇게 받은 인증번호를 전달해 휴대 전화에 입력하면 다른 사람의 모바일 건강보험증이 문제 없이 설치됩니다입을 맞춘다면 다른 사람의 모바일 건강보험증으로 진료를 받을 수 있는 겁니다 이뿐만이아닙니다 한 명에 모바일 건강보험증을 둠각자의 휴대전화에 설치하는 것도 가능했습니다 한 사람의 모발 건강보험증으로 여러 명이 진료 받는 것도 가능할 수 있다는 얘깁니다보험 중엔 사진이 없다 보니 병의원에서 본인 여부를 확인할 방법도 마땅치 않습니다 의료 보험이 없다옆에 사람 QR 신분증만 빌리면 된다 공기계 나 그걸 깔아 와서 그거만 내밀면 사실 저희는 죽을 수밖에 없는

In [5]:
# Step 2: 텍스트 전처리
try:
    processed_text = preprocess_text(transcript)
    print("Step 2: 전처리된 텍스트:", processed_text)
except Exception as e:
    print(f"Error: {e}")

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



Number of sentences: 82
Step 2: 전처리된 텍스트: ['여러분 안녕하십니까', '오늘은 저희가 단독 취재한 내용으로 뉴스 시작하겠습니다', '다음 주 월요일부터는 병원에 갈 때신분증을 꼭 챙겨야 합니다', '환자가 건강보험 자격이 있는지 혹시 다른 사람 명의로 진료받는 건 아닌지 병원이 확인하기 위해서입니다', '신분증 대신 휴대전화에 모바일 건강보험증을 설치해서 그걸 병원에 보여 줘도 됩니다', '그런데이 모바일다른 사람의 휴대전화 해도 쉽게 설치할 수 있고 또 그걸 병원이 적발하기 어렵다는 사실이 저희 취재 결과 확인됐습니다', '기자의 단독 보도입니다', '신분증 지참 필수 란 안내 포스터가 붙은 한내과의원 의원 협조를 미리 받아모바일 건강보험증으로 진료 접수를 해 봤습니다', '건강보험증  코드를 병원 기기로 인식하자 건강 보험 자격 확인이 되고 문제 없이완료됐습니다', '하지만 기자가 제시한 모바일 건강보험증은 본인이 아닌 동료의 것입니다', '타인 명의의 건강보험증을 병원에 제출했는데걸러내지 못한 겁니다', '어떻게 이게 가능한걸까 모바일 건강보험증 앱은 휴대 전화 번호를 입력한뒤 인증번호를 받아 본인임을 확인하는설치할 수 있습니다', '그런데 두 사람이 미리 짜고 상대방 휴대전화에 인증번호를 받을 사람의 전화 번호를 입력하고그렇게 받은 인증번호를 전달해 휴대 전화에 입력하면 다른 사람의 모바일 건강보험증이 문제 없이 설치됩니다', '입을 맞춘다면 다른 사람의 모바일 건강보험증으로 진료를 받을 수 있는 겁니다', '이뿐만이아닙니다', '한 명에 모바일 건강보험증을 둠각자의 휴대전화에 설치하는 것도 가능했습니다', '한 사람의 모발 건강보험증으로 여러 명이 진료 받는 것도 가능할 수 있다는 얘깁니다', '보험 중엔 사진이 없다 보니 병의원에서 본인 여부를 확인할 방법도 마땅치 않습니다', '의료 보험이 없다옆에 사람  신분증만 빌리면 된다 공기계 나 그걸 깔아 와서 그거만 내밀면 사실 저희는 죽을 수밖에 없는 시스템이거든요', 

In [6]:
# Step 3: 주제 추출 및 요약
topics = extract_topics_and_summarize(processed_text)
print("Extracted Topics and Summaries:\n", topics)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


[Kss]: From C:\Users\jwoo3\anaconda3\lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Topics extracted and summarized
Extracted Topics and Summaries:
 {'주제 1': '은지 병원이 확인하기 위해서 입니다 병원이 확인하기 위해서입니다 신분증 지참 필수 란 안내 포스터가 붙은 한내과의원 의원 협조를 미리 받아모바일 건강보험 증으로 진료 접수를 해 봤습니다 건강보험증  코', '주제 2': '많 많도 원칙적으로는 맞는 말해도 아니라는 대응이라는 지적을 피할 수는 없습니다 원칙적으로는 맞는 말해도 아니라는 대응이라는 지적을 피할 수는 없습니다 이런 허술한 본인 강화 제도로 이런 사례를 얼마나 줄일 수 있을지도 의문이 나 있을', '주제 3': '됐 됐 왔습니다 대명시장 백화점 기자 나와 있습니다 이런 것들을 정도는 전혀 사전에못 하고 있던 건가요 이렇게 밝혔습니다 종합적으로 문제가 뭔지 파악을 해서 대책마련을 하는 게 필요해 보입니다 먹으면은 다른 사람이진료', '주제 4': '올해는 특히 마늘이 걱정입니다 잦은 비 와부족한 일조량 때문에 상품성이 떨어지는 마늘이 많다고 합니다 수확한 마늘을 말리는 작업이 한창인 제주의 마늘 밭입니다 마늘 생육', '주제 5': '등이 등 병원이 적발하기 적발하기 적발하기 적발하기 적발하기 적발하기 적발하기 적발하기 적발하기 확인됐습니다 하지만 기자가 제시한 모바일 건강보험증은 본인이 아닌 동료의 것입니다 어떻게 이게 가능한걸까 모바일 건강보험증 앱은'}


In [9]:
# 이 함수 호출부를 유지하고 실행하여 오류를 확인합니다.
try:
    print("\n추출된 주제 목록:")
    for idx, (topic, summary) in enumerate(topics.items(), start=1):
        print(f"{idx}. {summary}")

    selected_topic_idx = int(input("\n원하는 주제의 번호를 선택하세요: ")) - 1
    selected_topic = list(topics.keys())[selected_topic_idx]
    selected_summary = topics[selected_topic]
    print(f"\n선택된 주제: {selected_topic}")

    # Step 4: 주제 관련 문장 찾기 및 영상 자르기
    relevant_sentences, duration = find_relevant_sentences(processed_text, selected_summary)
    if not relevant_sentences:
        raise ValueError(f"선택된 주제 '{selected_topic}'와 관련된 문장을 찾을 수 없습니다.")
    print(f"Relevant sentences for topic '{selected_topic}': {relevant_sentences}")
    print(f"Total duration: {duration} seconds")

    # 중요 프레임 추출 (1분 내의 영상만을 대상으로)
    start_time = 0  # 예시 시작 시간 (초 단위)
    end_time = start_time + duration  # 실제 문장의 길이를 시간으로 환산
    timestamps, face_centers = extract_important_frames(video_file, start_time, end_time)
    focus_frame_index = np.argmax(face_centers)
    focus_frame = face_centers[focus_frame_index]  # 가장 중요한 프레임 선택

    print(f"Focus frame: {focus_frame}")  # Debugging line

    # 관련 문장에 해당하는 영상을 자르기
    output_file = 'videos/shorts_output.mp4'
    cut_video_with_focus(video_file, start_time, end_time, output_file, face_centers)
    print(f"Video cut and saved to {output_file}")
except Exception as e:
    print(f"Error: {e}")


추출된 주제 목록:
1. 은지 병원이 확인하기 위해서 입니다 병원이 확인하기 위해서입니다 신분증 지참 필수 란 안내 포스터가 붙은 한내과의원 의원 협조를 미리 받아모바일 건강보험 증으로 진료 접수를 해 봤습니다 건강보험증  코
2. 많 많도 원칙적으로는 맞는 말해도 아니라는 대응이라는 지적을 피할 수는 없습니다 원칙적으로는 맞는 말해도 아니라는 대응이라는 지적을 피할 수는 없습니다 이런 허술한 본인 강화 제도로 이런 사례를 얼마나 줄일 수 있을지도 의문이 나 있을
3. 됐 됐 왔습니다 대명시장 백화점 기자 나와 있습니다 이런 것들을 정도는 전혀 사전에못 하고 있던 건가요 이렇게 밝혔습니다 종합적으로 문제가 뭔지 파악을 해서 대책마련을 하는 게 필요해 보입니다 먹으면은 다른 사람이진료
4. 올해는 특히 마늘이 걱정입니다 잦은 비 와부족한 일조량 때문에 상품성이 떨어지는 마늘이 많다고 합니다 수확한 마늘을 말리는 작업이 한창인 제주의 마늘 밭입니다 마늘 생육
5. 등이 등 병원이 적발하기 적발하기 적발하기 적발하기 적발하기 적발하기 적발하기 적발하기 적발하기 확인됐습니다 하지만 기자가 제시한 모바일 건강보험증은 본인이 아닌 동료의 것입니다 어떻게 이게 가능한걸까 모바일 건강보험증 앱은

선택된 주제: 주제 3
Relevant sentences for topic '주제 3': ['환자가 건강보험 자격이 있는지 혹시 다른 사람 명의로 진료받는 건 아닌지 병원이 확인하기 위해서입니다', '그런데이 모바일다른 사람의 휴대전화 해도 쉽게 설치할 수 있고 또 그걸 병원이 적발하기 어렵다는 사실이 저희 취재 결과 확인됐습니다', '기자의 단독 보도입니다', '신분증 지참 필수 란 안내 포스터가 붙은 한내과의원 의원 협조를 미리 받아모바일 건강보험증으로 진료 접수를 해 봤습니다', '건강보험증  코드를 병원 기기로 인식하자 건강 보험 자격 확인이 되고 문제 없이완료됐습니다', '하지만 기자가 제시한 모바일 건강보험증은 본인이 아닌 동료의 것입니다', '타인 명의의 건강보험증을

870it [04:01,  3.60it/s]                         


Extracted face centers: [900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 900, 1695, 1695, 1653, 1585, 1557, 949, 920, 1057, 900, 1057, 880, 840, 1000, 820, 801, 801, 781, 761, 741, 741, 721, 701, 698, 681, 674, 661, 650, 854, 846, 621, 601, 818, 804, 791, 785, 514, 514, 509, 500, 500, 497, 486, 486, 486, 474, 474, 463, 463, 459, 451, 445, 440, 433, 428, 428, 417, 417, 417, 405, 405, 394, 394, 394, 394, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 394, 394, 394, 394, 394, 394, 394, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 385, 1407, 1403, 1384, 1384, 1384, 1384, 1384, 1384, 1384, 1393, 1393, 1393, 1393,

MoviePy - Done.
Moviepy - Writing video videos/shorts_output.mp4


Moviepy - Done !
Moviepy - video ready videos/shorts_output.mp4
Video cut and saved to videos/shorts_output.mp4
